In [1]:
## This scripts serves as the extractor for all features used in this project
## it will iterate through each image in the dataset and extract features


In [5]:
# import modules
import pandas as pd
import numpy as np
import cv2 as cv
from radiomics import featureextractor

In [23]:
# Load training data csv. The original file contains the images names and patient data
training_data_csv = pd.read_csv('../merged_training_data.csv')
training_data_csv.head()

,image_id,age_approximate,sex,melanoma,seborrheic_keratosis
0,ISIC_0000000,55,female,0.0,0.0
1,ISIC_0000001,30,female,0.0,0.0
2,ISIC_0000002,60,female,1.0,0.0
3,ISIC_0000003,30,male,0.0,0.0
4,ISIC_0000004,80,male,1.0,0.0


In [34]:
for i in training_data_csv['image_id']:
    print(type(i))
  
    # Load an image and mask
    print('../../ISIC-2017_Training_Data/'+ i)
    img = cv.imread('../../ISIC-2017_Training_Data/'+ str(i),-1) 
    mask = cv.imread('../../ISIC-2017_Training_Part1_GroundTruth/'+i+'_segmentation.png',-1)
    # Color conversion and normalization
    output_array = np.zeros((img.shape[0],img.shape[1])) # Array to store normalized image
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

<class 'str'>
../../ISIC-2017_Training_Data/ISIC_0000000


[ WARN:0@1620.862] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../../ISIC-2017_Training_Data/ISIC_0000000'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [17]:
374/1626

0.23001230012300122